In [1]:
import pandas as pd

In [22]:
data = [[121, 'US', 'approved', 1000, '2018-12-18'], [122, 'US', 'declined', 2000, '2018-12-19'], [123, 'US', 'approved', 2000, '2019-01-01'], [124, 'DE', 'approved', 2000, '2019-01-07']]
transactions = pd.DataFrame(data, columns=['id', 'country', 'state', 'amount', 'trans_date']).astype({'id':'Int64', 'country':'object', 'state':'object', 'amount':'Int64', 'trans_date':'datetime64[ns]'})

In [52]:
df = transactions.copy()

In [53]:
df.head()

,id,country,state,amount,trans_date
0,121,US,approved,1000,2018-12-18
1,122,US,declined,2000,2018-12-19
2,123,US,approved,2000,2019-01-01
3,124,DE,approved,2000,2019-01-07


In [54]:
df["year_month"] = df["trans_date"].dt.to_period("M")

In [55]:
df["year_month"]

0    2018-12
1    2018-12
2    2019-01
3    2019-01
Name: year_month, dtype: period[M]

In [56]:
df.head()

,id,country,state,amount,trans_date,year_month
0,121,US,approved,1000,2018-12-18,2018-12
1,122,US,declined,2000,2018-12-19,2018-12
2,123,US,approved,2000,2019-01-01,2019-01
3,124,DE,approved,2000,2019-01-07,2019-01


In [57]:
df_aprv = df[df["state"] == "approved"]
df_de = df[df["state"] == "declined"]

In [58]:
df_count_aprv = df_aprv.groupby(["year_month", "country"]).agg({"amount": ["sum", "count"]}).reset_index()
df_count_aprv.columns = ["month", "country",  "approved_total_amount", "approved_count"]
df_count_aprv.head()

,month,country,approved_total_amount,approved_count
0,2018-12,US,1000,1
1,2019-01,DE,2000,1
2,2019-01,US,2000,1


In [59]:
df_count_total = df.groupby(["year_month", "country"]).agg({"amount": ["sum", "count"]}).reset_index()
df_count_total.columns = ["month", "country",  "trans_total_amount", "trans_count"]
df_count_total.head()

,month,country,trans_total_amount,trans_count
0,2018-12,US,3000,2
1,2019-01,DE,2000,1
2,2019-01,US,2000,1


In [60]:
df_count_aprv

,month,country,approved_total_amount,approved_count
0,2018-12,US,1000,1
1,2019-01,DE,2000,1
2,2019-01,US,2000,1


In [63]:
df_merge = pd.merge(df_count_total, df_count_aprv, on=["month", "country"], how="left")
df_merge.head()

,month,country,trans_total_amount,trans_count,approved_total_amount,approved_count
0,2018-12,US,3000,2,1000,1
1,2019-01,DE,2000,1,2000,1
2,2019-01,US,2000,1,2000,1


In [64]:
df_result = df_merge[["month", "country", "trans_count", "approved_count", "trans_total_amount", "approved_total_amount"]]
# | month    | country | trans_count | approved_count | trans_total_amount | approved_total_amount |

In [65]:
df_result.head()

,month,country,trans_count,approved_count,trans_total_amount,approved_total_amount
0,2018-12,US,2,1,3000,1000
1,2019-01,DE,1,1,2000,2000
2,2019-01,US,1,1,2000,2000


In [72]:
import pandas as pd

def monthly_transactions(transactions: pd.DataFrame) -> pd.DataFrame:
    df = transactions
    df["month"] = df["trans_date"].dt.to_period("M")
    df_aprv = df[df["state"] == "approved"]
    df_count_aprv = df_aprv.groupby(["month", "country"]).agg({"amount": ["sum", "count"]}).reset_index()
    df_count_aprv.columns = ["month", "country",  "approved_total_amount", "approved_count"]
    df_count_total = df.groupby(["month", "country"]).agg({"amount": ["sum", "count"]}).reset_index()
    df_count_total.columns = ["month", "country",  "trans_total_amount", "trans_count"]
    df_merge = pd.merge(df_count_total, df_count_aprv, on=["month", "country"], how="left")
    df_result = df_merge[["month", "country", "trans_count", "approved_count", "trans_total_amount", "approved_total_amount"]]
    return df_result

In [73]:
monthly_transactions(transactions)

,month,country,trans_count,approved_count,trans_total_amount,approved_total_amount
0,2018-12,US,2,1,3000,1000
1,2019-01,DE,1,1,2000,2000
2,2019-01,US,1,1,2000,2000


In [74]:
import pandas as pd

# def monthly_transactions(transactions: pd.DataFrame) -> pd.DataFrame:
#     df = transactions
#     return transactions

def monthly_transactions(transactions: pd.DataFrame) -> pd.DataFrame:
    df = transactions
    df["country"] = df["country"]
    df["month"] = df["trans_date"].dt.to_period("M").astype("str")
    df_aprv = df[df["state"] == "approved"]
    df_count_aprv = df_aprv.groupby(["month", "country"], dropna=False).agg({"amount": ["sum", "count"]}).reset_index()
    df_count_aprv.columns = ["month", "country",  "approved_total_amount", "approved_count"]
    df_count_total = df.groupby(["month", "country"], dropna=False).agg({"amount": ["sum", "count"]}).reset_index()
    df_count_total.columns = ["month", "country",  "trans_total_amount", "trans_count"]
    df_merge = pd.merge(df_count_total, df_count_aprv, on=["month", "country"], how="outer")
    df_merge = df_merge[["month", "country", "trans_count", "approved_count", "trans_total_amount", "approved_total_amount"]]
    df_merge["trans_count"] = df_merge["trans_count"].fillna(0)
    df_merge["approved_count"] = df_merge["approved_count"].fillna(0)
    df_merge["trans_total_amount"] = df_merge["trans_total_amount"].fillna(0)
    df_merge["approved_total_amount"] = df_merge["approved_total_amount"].fillna(0)
    return df_merge

In [76]:
res = monthly_transactions(transactions)

In [77]:
type(res)

pandas.core.frame.DataFrame

In [78]:
res.head()

,month,country,trans_count,approved_count,trans_total_amount,approved_total_amount
0,2018-12,US,2,1,3000,1000
1,2019-01,DE,1,1,2000,2000
2,2019-01,US,1,1,2000,2000


In [79]:
import pandas as pd

# Create a list of data
data = [
    ["2018-12", "US", 2, 1, 3000, 1000],
    ["2019-01", "US", 1, 1, 2000, 2000],
    ["2019-01", None, 1, 1, 2000, 2000],
]

# Create DataFrame
df = pd.DataFrame(data, columns=["month", "country", "trans_count", "approved_count", "trans_total_amount", "approved_total_amount"])

# Print the DataFrame
print(df)


     month country  trans_count  approved_count  trans_total_amount  \
0  2018-12      US            2               1                3000   
1  2019-01      US            1               1                2000   
2  2019-01    None            1               1                2000   

   approved_total_amount  
0                   1000  
1                   2000  
2                   2000  


In [81]:
import pandas as pd

# Create a list of data
data = [
    {"id": 121, "country": "US", "state": "approved", "amount": 1000, "trans_date": "2018-12-18"},
    {"id": 122, "country": "US", "state": "declined", "amount": 2000, "trans_date": "2018-12-19"},
    {"id": 123, "country": "US", "state": "approved", "amount": 2000, "trans_date": "2019-01-01"},
    {"id": 124, "country": None, "state": "approved", "amount": 2000, "trans_date": "2019-01-07"},
]

# Create DataFrame
df = pd.DataFrame(data)

# Print the DataFrame
print(df)


    id country     state  amount  trans_date
0  121      US  approved    1000  2018-12-18
1  122      US  declined    2000  2018-12-19
2  123      US  approved    2000  2019-01-01
3  124    None  approved    2000  2019-01-07


In [86]:
df["month"] = pd.to_datetime(df["trans_date"]).dt.to_period("M").astype("str")
df_aprv = df[df["state"] == "approved"]
df_count_aprv = df_aprv.groupby(["month", "country"], dropna=False).agg({"amount": ["sum", "count"]}).reset_index()

In [87]:
df_count_aprv

month country amount      
                      sum count
0  2018-12      US   1000     1
1  2019-01      US   2000     1
2  2019-01     NaN   2000     1

In [85]:
df_aprv

,id,country,state,amount,trans_date,month
0,121,US,approved,1000,2018-12-18,2018-12
2,123,US,approved,2000,2019-01-01,2019-01
3,124,None,approved,2000,2019-01-07,2019-01
